In [54]:
import json
import nltk
import inflect
import tqdm
engine = inflect.engine()
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/luke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/luke/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
recipes = json.load(open("../data/1M/recipe1M_layers/layer1.json"))

In [66]:
units = ["bushel", "cup", "dash", "drop", "fl." "oz", "g", "gallon", "glass",
"kg", "liter", "ml", "ounce", "c.", "pinch", "pint", "pound", "lb", "quart",
"scoop", "shot", "tablespoon", "teaspoon", "tsp", "tbsp"]

measurable_indices = []
measurables = 0
nb_to_try = len(recipes)

for e, r in tqdm.tqdm_notebook(enumerate(recipes[:nb_to_try])) :
    
    contains_immesurable = False
    
    for ingredient in r['ingredients'] :
               
        
        tags = nltk.pos_tag(nltk.word_tokenize(ingredient['text']))
        if ((len(tags) > 2) and (tags[0][1] == 'CD')) :
            
            unit_to_recognize = tags[1][0].lower()

            singularized = engine.singular_noun(unit_to_recognize)
            
            if singularized != False :
                unit_to_recognize = singularized

            quantity = tags[0][0] + " " + unit_to_recognize
            
            if (not unit_to_recognize in units) :
                contains_immesurable = True
                break

    if not contains_immesurable :
        measurables += 1
        measurable_indices.append(e)
        
print(str(measurables) + " recipes contained only mesurable ingredients => " + str(100*measurables/nb_to_try) + "%")

                

115807 recipes contained only mesurable ingredients => 11.246455347084645%


In [70]:
det_ingr = json.load(open("../data/1M/det_ingr.json"))

In [81]:
det_ingr[0]['valid'].index(False)

8

In [83]:
usda_mappable_indices = []
usda_mappable = 0

for index in tqdm.tqdm_notebook(measurable_indices) :
    try :
        det_ingr[index]['valid'].index(False)
    except :
        usda_mappable += 1
        usda_mappable_indices.append(index)

print(str(usda_mappable) + " recipes contained only mappable ingredients => " + str(100*usda_mappable/len(measurable_indices)) + "%")


107977 recipes contained only mappable ingredients => 93.23875068001071%
